In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchvision.models as models
import random as random
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset
from functools import reduce
from operator import __or__
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from PIL import Image

In [2]:
# alignmnist = np.load('data/alignmnist500.npz')
# x = alignmnist['x']
# y = alignmnist['y']

In [3]:
class AlignMNIST(Dataset):
    def __init__(self, path, transforms=None, train=True):
        if train:
            self.data = np.load(path, mmap_mode='r')
        else:
            self.data = np.load(path)
        self.transforms = transforms
        
        
    def __getitem__(self, index):
        img, label = self.data['x'][index], self.data['y'][index]
        img = Image.fromarray(img, mode='L')
        if self.transforms is not None:
            img = self.transforms(img)
        return (img, int(label))

    def __len__(self):
        return self.data['x'].shape[0] # of how many data(images?) you have

In [4]:
n_labels = 10
cuda = torch.cuda.is_available()
device = "cuda:0" if cuda else "cpu"

def onehot(k):
    """
    Converts a number to its one-hot or 1-of-k representation
    vector.
    :param k: (int) length of vector
    :return: onehot function
    """
    def encode(label):
        y = torch.zeros(k)
        if label < k:
            y[label] = 1
        return y
    return encode

compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        ])

def get_mnist(location="./", batch_size=128):
    
    
    
    img_transform = lambda x: compose(x).view(-1)
    

    mnist_train = AlignMNIST('data/alignmnist500.npz', img_transform, train=True)
    mnist_valid = AlignMNIST('data/alignmnist500.npz', img_transform, train=False)

    # Dataloaders for MNIST
    train = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size)
    
    validation = torch.utils.data.DataLoader(mnist_valid, batch_size=batch_size)

    return train, validation

In [5]:
train_loader, valid_loader = get_mnist()

In [6]:
image, label = next(iter(train_loader))
plt.imshow(image[0].view(28,28))

MemoryError: 

In [ ]:
class Classifier(nn.Module):
    def __init__(self, in_dim=784):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_features=in_dim, out_features=256),
                                 nn.ReLU(),
#                                  nn.Dropout(0.3),
                                 nn.Linear(in_features=256, out_features=256),
                                 nn.ReLU(),
#                                  nn.Dropout(0.3),
                                 nn.Linear(in_features=256, out_features=256),
                                 nn.ReLU(),
#                                  nn.Dropout(0.3),
                                 nn.Linear(in_features=256, out_features=10),
                                 nn.LogSoftmax(dim=-1))
        
    def forward(self,x):        
        return self.net(x)

In [ ]:
classifier = Classifier().to(device)

In [ ]:
optimizerC = torch.optim.Adam(classifier.parameters(), lr=0.00005)

In [ ]:
BCE = nn.BCELoss()
NLL = nn.NLLLoss()

MSE = nn.MSELoss()
NUM_EPOCH = 2

def train():
    loss_c = []
    for epoch in range(NUM_EPOCH):
        for idx, batch_data in enumerate(train_loader):
            batch_size = batch_data[0].shape[0]            
            
            # retrieve real data from loader
            real_images =batch_data[0].view(batch_size, -1).cuda()
#             print(real_images)
            classes_real = batch_data[1].cuda()
            
            # Train classifier on fake and real data
            prediction_real = classifier(real_images)
            
            
            
#             lossC = MSE(prediction_real, classes_real)
#             print(classes_real)
            lossC = NLL(prediction_real, classes_real)
            
            lossC.backward()
            
            optimizerC.step()           
            loss_c.append(lossC.item())
            if idx %50 == 0:
                print("Updates: ", idx, " Closs: ", np.mean(loss_c[-20:]))
            if idx%400 == 0:
                print("Validation accuracy:", test())
            

In [ ]:
def test():
    r =0
    length = 0 
    for idx, batch_data in enumerate(valid_loader):
        batch_size = batch_data[0].shape[0]
        real_images = batch_data[0].cuda()
        classes_real = batch_data[1].cuda()
#         print(classes_real.argmax(dim=-1))

        predictions_real = classifier(real_images).argmax(dim=-1)
#         print(predictions_real)
#         r += torch.sum(abs(predictions_real-classes_real)).item()
        r += predictions_real.eq(classes_real).sum().item()
        length += batch_size
    return r/length

In [ ]:
train()